In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from data_generation import tbp_util
from data_generation.tbp_energy_calculations import visualize_dataset

In [ ]:
from data_generation import matplotlib_style

matplotlib_style.init_plt_style()

In [ ]:
# todo read from config file
# also see tbp_util.py
G = 1.0
M = np.array([1.0, 1.0, 1.0])

In [ ]:
# for dataset, x, y, vx, vy in tbp_util.load_datasets(load_successful=False, load_unsuccessful=True, limit=1):
#     visualize_dataset(x, y, vx, vy)
#
visualize_dataset(*tbp_util.load_dataset("15"), G, M)

In [ ]:
x, y, vx, vy = tbp_util.load_dataset("50000")
x[0, :]

In [ ]:
y[0, :]

In [ ]:
dx = x[:-1] - x[1:,]
dx = dx * 1000000
plt.figure()
plt.boxplot(dx, showfliers=False)
plt.show()

In [ ]:
from tensorflow import keras

keras.backend.set_floatx('float64')
keras.backend.floatx()

Load training data

In [ ]:
x_train = np.ndarray((0, 12), dtype=np.float64)
y_train = np.ndarray((0, 12), dtype=np.float64)

for dataset, x, y, vx, vy in tbp_util.load_datasets(limit=100):
    input_data = np.column_stack((x, y, vx, vy))
    output_data = input_data.copy()

    # predict 1000 samples into the future and delete 9/10 training examples to speed-up training
    input_data = input_data[:-1000:10, ]
    output_data = output_data[1000::10, ]

    x_train = np.concatenate((x_train, input_data))
    y_train = np.concatenate((y_train, output_data))


first training example **[ x1, x2, x3, y1, y2, y3, vx1, vx2, vx3, vy1, vy2, vy3 ]**
first testing  example **[ x1', x2', x3', y1', y2', y3', vx1', vx2', vx3', vy1', vy2', vy3' ]**

In [ ]:
x_train[0,]

In [ ]:
y_train[0,]

In [ ]:
batch_size = 256
epochs = 256
validation_split = 0.1
steps_per_epoch = round((x_train.shape[0] * (1 - validation_split)) / batch_size)

# try to keep it above 10^-8
learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_rate=0.75,
    decay_steps=steps_per_epoch * 10
)
plt.figure()
plt.yscale('log')
plt.plot([learning_rate_schedule(epoch * steps_per_epoch) for epoch in range(epochs)])
plt.show()

In [ ]:
batch_size = 256
epochs = 256
validation_split = 0.1
steps_per_epoch = round((x_train.shape[0] * (1 - validation_split)) / batch_size)

boundaries = np.array([10]) * steps_per_epoch

values = [0.0001, 0.00001]
learning_rate_schedule = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries.tolist(), values)
plt.figure()
plt.yscale('log')
plt.plot([learning_rate_schedule(epoch * steps_per_epoch) for epoch in range(epochs)])
plt.show()

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation=keras.activations.relu, input_shape=[12]),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(64, activation=keras.activations.relu),
    keras.layers.Dense(12, activation=keras.activations.linear)
])

model.compile(
    # keras.optimizers.Adam(learning_rate=learning_rate_schedule),
    keras.optimizers.Adam(learning_rate=0.000001),
    loss='mae',
    metrics=['mae', 'mse']
)

X: 1.000000000
Y: 1.000000001
Y: 0.000000001 * 1_000_000

In [ ]:
hist_callback = keras.callbacks.History()
callbacks = [
    hist_callback
]

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks
)

In [ ]:
plt.figure()
plt.plot(hist_callback.history['loss'])
plt.show()

In [ ]:
dataset_to_predict = '20006'
x, y, vx, vy = tbp_util.load_dataset(dataset_to_predict)
print(x.shape)

In [ ]:
length_to_predict = 5000000
x = x[0:length_to_predict, ]
y = y[0:length_to_predict, ]
vx = vx[0:length_to_predict, ]
vy = vy[0:length_to_predict, ]
visualize_dataset(x, y, vx, vy, 1000)

In [ ]:

limit = round(length_to_predict / 1000)
limit = 1000
y_pred = np.zeros((limit, 12), dtype=np.float64)
y_pred[0,] = np.concatenate((x[0,], y[0,], vx[0,], vy[0,]))

for i in range(limit - 1):
    prediction = model(y_pred[i,].reshape(1, 12), training=False).numpy()
    y_pred[i + 1,] = prediction

In [ ]:
x, y, vx, vy = np.hsplit(y_pred, 4)
# x, y, vx, vy = np.hsplit(x_train[:25000], 4)
visualize_dataset(x, y, vx, vy, down_sample_factor=1)

In [ ]:
visualize_dataset(*tbp_util.load_dataset('2'))

In [ ]:
model.save()

In [ ]:
for x in ["h5", "keras", "tf"]:
    model.save(f"model.{x}", save_format=x)

In [ ]:
for x in ["h5", "tf"]:
    model.save_weights(f"model_weights.{x}", save_format=x)

In [ ]:
model.load_weights("model_weights.h5")